<a href="https://colab.research.google.com/github/Firojpaudel/Machine-Learning-Notes/blob/main/Practical%20Deep%20Learning%20For%20Coders/Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MultiLabel Classification And Regression


In the last chapter, we learned some practical tips for training models, like choosing the right learning rates and number of epochs which were really important for good results. Now, this chapter dives into two new types of computer vision problems: multi-label classification (predicting multiple labels for an image or none at all) and regression (predicting numbers instead of categories). Along the way, we'll explore output activations, targets, and loss functions in more depth.

## MultiLabel Classification

*MultiLabel Classification* refers to the problem of identifying the objects in images that may not contain exactly one type of object.

There may be more than one type of object or there may be none.

In previous bear classifier we built, it had no ability to predict "not bear at all". So, this time we are trying to fix that.

First, let's see what a mutilabel dataset looks like. But before that lets set up the notebook.

In [1]:
### Setting up the notebook
##@ Notebook initialization
%reload_ext autoreload
%autoreload 2
%matplotlib inline

##@ Installing dependencies
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

##@ Importing the necessary libraries
from fastbook import *
from fastai.callback.fp16 import *
from fastai.vision.all import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Mounted at /content/gdrive


### Getting Data

We will get the PASCAL datset which has more than one kind of classified object per image.

In [2]:
##@ Getting the PASCAL dataset
path = untar_data(URLs.PASCAL_2007)

<details>
<summary><b>More about this dataset:</b></summary>
This dataset is a bit different from the ones we've worked with before. Instead of being organized by filenames or folders, it comes with a CSV file that tells us which labels to use for each image.
</details>


In [3]:
##@ inspecting the dataset
df = pd.read_csv(path/'train.csv')
df.head()

,fname,labels,is_valid
0,000005.jpg,chair,True
1,000007.jpg,car,True
2,000009.jpg,horse person,True
3,000012.jpg,car,False
4,000016.jpg,bicycle,True


In [4]:
header = """
===============================================================
                Pandas and DataFrames Subsection in Book
===============================================================
"""
print(header)

# Accessing rows and columns using .iloc
print("------- Using .iloc: -------")
print(f"Returning the 1st column of the dataframe:\n\n{df.iloc[:, 0]} \n")  # 1st column (labels)
print(f"Returning the 1st row of the dataframe:\n\n{df.iloc[0, :]} \n")    # 1st row (details of item1)

# Grabbing a column by name (without using iloc)
print(f"Grabbing the column 'labels' directly from the dataframe:\n\n{df['labels']}\n\n")

# Calculations in the dataframe
print("----- Calculations in the DataFrame itself ------ \n")
print("Creating a new dataframe for this one:\n")
df1 = pd.DataFrame({
    'Items Sold': [5, 2, 5, 4, 9],
    'Rates': [100, 50, 60, 400, 1000]
})
print(f"The new dataframe before the calculation:\n\n{df1}\n")
df1['Income'] = df1['Items Sold'] * df1['Rates']  # Calculating income as Items Sold * Rates

print(f"The new dataframe after the calculation:\n\n{df1}\n")


                Pandas and DataFrames Subsection in Book

------- Using .iloc: -------
Returning the 1st column of the dataframe:

0       000005.jpg
1       000007.jpg
2       000009.jpg
3       000012.jpg
4       000016.jpg
           ...    
5006    009954.jpg
5007    009955.jpg
5008    009958.jpg
5009    009959.jpg
5010    009961.jpg
Name: fname, Length: 5011, dtype: object 

Returning the 1st row of the dataframe:

fname       000005.jpg
labels           chair
is_valid          True
Name: 0, dtype: object 

Grabbing the column 'labels' directly from the dataframe:

0                chair
1                  car
2         horse person
3                  car
4              bicycle
             ...      
5006      horse person
5007              boat
5008    person bicycle
5009               car
5010               dog
Name: labels, Length: 5011, dtype: object


----- Calculations in the DataFrame itself ------ 

Creating a new dataframe for this one:

The new dataframe before the calc

### Constriucting a DataBlock

#### Key Concepts

Before diving into the DataBlock construction, it is important to understand the two main classes used in PyTorch and FastAI to represent datasets and batches:

- **Dataset**: A collection that returns a tuple of the independent variable (input) and the dependent variable (target) for a single item.
- **DataLoader**: An iterator that provides a stream of mini-batches. Each mini-batch consists of a batch of inputs and a batch of targets.

FastAI adds two additional layers on top of these:

- **Datasets**: An iterator that contains both the training and validation datasets.
- **DataLoaders**: An object that contains a training DataLoader and a validation DataLoader.

A DataLoader is built on top of a Dataset to add functionalities like batching, shuffling, and parallel data loading.

#### Step-by-Step Construction of a DataBlock

##### **1. Starting Simple: DataBlock With No Parameters**

---
We begin my creating an empty ```DataBlock``` object with no parameters. This serves as the foundation for building the DataBlock step by step.

In [5]:
dblock = DataBlock()

Next, we create a ```Datasets``` object from this ```DataBlock```. The only required argument is the data source—in this case, a **DataFrame**.

In [6]:
dsets = dblock.datasets(df)

This creates a training and validation dataset, which can be accessed as:

In [7]:
dsets.train[0]

(fname       008663.jpg
 labels      car person
 is_valid         False
 Name: 4346, dtype: object,
 fname       008663.jpg
 labels      car person
 is_valid         False
 Name: 4346, dtype: object)

By default, the ```DataBlock``` assumes there are two items: the input (independent variable) and the target (dependent variable). This will simply return a row of the DataFrame.

##### **2. Specifying the Input Target with ```get_x``` and ```get_y```**

---

To handle the DataFrame more effectively, we specify which columns correspond to the input and target variables using ```get_x``` and ```get_y``` functions.

In [8]:
dblock = DataBlock(get_x = lambda r: r['fname'], get_y = lambda r: r['labels'])
dsets = dblock.datasets(df)
dsets.train[0]

('005620.jpg', 'aeroplane')

In [9]:
#@ Alternatively, we can define the functions more explicitly:
def get_x(r):
  return r['fname']

def get_y(r):
  return r['labels']

dblock = DataBlock(get_x = get_x, get_y = get_y)

Although using **lambda functions** is convenient, they are not compatible with serialization. For saving models after training, it's better to use the more verbose function definitions.